In [2]:
from sage.all import *
import sage.groups.matrix_gps.orthogonal

### Creating Random Unitary Matrices

Sage Reference: [Unitary Group $GU(n,q)$](https://doc.sagemath.org/html/en/reference/groups/sage/groups/matrix_gps/unitary.html)


**INPUT:**

- `n` – positive integer

- `R` – ring or an integer; if an integer is specified, the corresponding finite field is used

- `var` – (default: 'a') variable used to represent generator of the finite field, if needed

- `invariant_form` – (optional) instances being accepted by the matrix-constructor which define a square matrix over describing the hermitian form to be kept invariant by the unitary group; the form is checked to be non-degenerate and hermitian but not to be positive definite

**OUTPUT:** The general unitary group

### Symmetric Function

In [12]:
# Set up polynomial ring with variables t1, t2, t3 over Complex Field
R = PolynomialRing(ZZ, ['t1', 't2', 't3'])
t1, t2, t3 = R.gens()

Sym = SymmetricFunctions(ZZ)
e = Sym.e() # Elementary symmetric function
h = Sym.h() # Complete symmetric function
p = Sym.p() # Power sum symmetric function

# Expand each symmetric function
h2_exp = h[2].expand(3)(t1, t2, t3)
h3_exp = h[3].expand(3)(t1, t2, t3)

e2_exp = e[2].expand(3)(t1, t2, t3)
e3_exp = e[3].expand(3)(t1, t2, t3)

p2_exp = p[2].expand(3)(t1, t2, t3)
p3_exp = p[3].expand(3)(t1, t2, t3)

print("h_2 =", h2_exp)
print("h_3 =", h3_exp)
print("e_2 =", e2_exp)
print("e_3 =", e3_exp)
print("p_2 =", p2_exp)
print("p_3 =", p3_exp)

h_2 = t1^2 + t1*t2 + t2^2 + t1*t3 + t2*t3 + t3^2
h_3 = t1^3 + t1^2*t2 + t1*t2^2 + t2^3 + t1^2*t3 + t1*t2*t3 + t2^2*t3 + t1*t3^2 + t2*t3^2 + t3^3
e_2 = t1*t2 + t1*t3 + t2*t3
e_3 = t1*t2*t3
p_2 = t1^2 + t2^2 + t3^2
p_3 = t1^3 + t2^3 + t3^3


### Commutative Monoid Ring

In [14]:
# Setup: Symmetric functions and ring
Sym = SymmetricFunctions(QQ)
h = Sym.h()
e = Sym.e()
p = Sym.p()

# Define a commutative monoid M
class MonoidElement:
    def __init__(self, label):
        self.label = label  # could be tuple, string, integer, etc.

    def __mul__(self, other):
        # define how two elements multiply
        return MonoidElement(f"({self.label} * {other.label})")

    def __pow__(self, k):
        # define m^k
        result = self
        for _ in range(k - 1):
            result = result * self
        return result

    def __repr__(self):
        return str(self.label)

In [15]:
# Example: a monoid with labeled elements
m = MonoidElement('m')

# R[M] element represented as dict {MonoidElement: R-coefficient}
def monoid_ring_element(coeff, m):
    return {m: coeff}

# Plethystic action on R[M]
def plethysm_hk_on_RM(hk, coeff, m):
    mk = m ** hk.degree()
    return {mk: hk.plethysm(coeff)}

def plethysm_ek_on_RM(ek, coeff, m):
    mk = m ** ek.degree()
    return {mk: ek.plethysm(coeff)}

# If R is torsion-free: implement the pk plethysm sum
def plethysm_pk_on_function(pk, f_dict, target_m, monoid_elements):
    k = pk.degree()
    result = 0
    for n in monoid_elements:
        if n ** k == target_m:
            result += pk.plethysm(f_dict.get(n, 0))
    return result

In [16]:
# Example usage
a = h[1] + 2*h[2]  # an example element in R (symmetric functions)
hk = h[2]
ek = e[2]
pk = p[2]

print("h_k ◦ (a[m]) =", plethysm_hk_on_RM(hk, a, m))
print("e_k ◦ (a[m]) =", plethysm_ek_on_RM(ek, a, m))

# For p_k action on f: assume M = {m, m2, m3}, simulate matching condition
m2 = m ** 2
m3 = m ** 3
f_dict = {
    m: a,
    m2: h[1],
    m3: e[1]
}
monoid_elements = [m, m2, m3]

target = m2  # compute (p_k ◦ f)(m2)
result = plethysm_pk_on_function(pk, f_dict, target, monoid_elements)
print(f"(p_k ◦ f)({target}) =", result)


h_k ◦ (a[m]) = {(m * m): h[2] + 2*h[2, 1] + 3*h[2, 2] - 2*h[3, 1] + 2*h[4]}
e_k ◦ (a[m]) = {(m * m): 2*e[1, 1, 1] + e[1, 1, 1, 1] + e[2] - 2*e[2, 1] - e[2, 2] - 2*e[3, 1] + 2*e[4]}
(p_k ◦ f)((m * m)) = 0


### Computing Λ-distributions

In [ ]:
s = SymmetricFunctions(QQ).schur()
e = SymmetricFunctions(QQ).elementary()

In [ ]:
f1 = s([Integer(2),Integer(1)]); f1
f2 = e(f1); f2 
f1 == f2
f1.expand(Integer(3), alphabet=['x','y','z'])
f2.expand(Integer(3), alphabet=['x','y','z'])

In [ ]:
G = GU(Integer(3), Integer(7)); G
G.gens()
GU(Integer(2), QQ)

In [ ]:
S = SymmetricGroup(3)
S.list()

In [ ]:
Sym = SymmetricFunctions(ZZ)
h = Sym.h()
e = Sym.e()
p = Sym.p()

# Pre-lambda ring structure on all Integers \mathbb{Z}
class PreLambdaRingZZ:
    def __init__(self):
        self.base_ring = ZZ

    def h_comp_n(self, k, n):
        """h_k o n = binomial(k + n - 1, n - 1)"""
        return binomial(k + n - 1, n - 1)

    def e_comp_n(self, k, n):
        """e_k o n = binomial(n, k)"""
        return binomial(n, k)

    def p_comp_n(self, k, n):
        """p_k o n = n (identity in ZZ)"""
        return n

    def verify_h_distributive_over_addition(self, k, r, s):
        """Verify that h_k o (r + s) = sum_{i = 0}^{k} h_{k-i}(r) * h_i(s)"""
        left = self.h_comp_n(k, r + s)
        right = sum(self.h_comp_n(k - i, r) * self.h_comp_n(i, s) for i in range(k + 1))
        return left == right

    def verify_h1_identity(self, r):
        """Check that h_1 o r = r"""
        return self.h_comp_n(1, r) == r

In [ ]:
# Instantiate and test
R = PreLambdaRingZZ()
n, m = 4, 3

# Verifications
print("h_1(n) == n:", R.verify_h1_identity(n))
print("h_2(n + m) == sum of h_i:", R.verify_h_distributive_over_addition(2, n, m))

# Example usage
for k in range(5):
    print(f"h_{k}({n}) =", R.h_comp_n(k, n))
    print(f"e_{k}({n}) =", R.e_comp_n(k, n))
    print(f"p_{k}({n}) =", R.p_comp_n(k, n))

## Cyclic Groups (C_n)

In [ ]:
from sage.groups.matrix_gps.finitely_generated import FinitelyGeneratedMatrixGroup_gap
from sage.matrix.constructor import matrix
from sage.rings.integer_ring import ZZ

def cyclic_group_matrix_representation(n):
    """
    Return the matrix representation of the cyclic group C_n as permutation matrices.
    
    INPUT:
    - n -- positive integer, the order of the cyclic group
    
    OUTPUT:
    - A matrix group isomorphic to C_n where the generator is a permutation matrix
      that cycles the basis vectors.
    
    EXAMPLES:
        sage: G = cyclic_group_matrix_representation(3)
        sage: G.order()
        3
        sage: G.is_abelian()
        True
    """
    if n <= 0:
        raise ValueError("n must be a positive integer")
    
    # Create the generator matrix - a cyclic permutation matrix
    def cyclic_perm_matrix(n):
        """Create an n×n cyclic permutation matrix"""
        M = matrix(ZZ, n, n)
        for i in range(n-1):
            M[i+1, i] = 1
        M[0, n-1] = 1
        return M
    
    generator = cyclic_perm_matrix(n)
    return FinitelyGeneratedMatrixGroup_gap([generator], check=False)

TypeError: MatrixGroup_gap.__init__() got an unexpected keyword argument 'check'